## Imports

In [1]:
!pip install sec-edgar-downloader

In [0]:
import sec_edgar_downloader
import google.colab as colab
import shutil
import os
import pandas as pd
import random
from IPython.display import display_html

import re
import requests
import unicodedata
from bs4 import BeautifulSoup
random.seed(716) 

### Mount Google Drive

In [0]:
def mount_google_drive():
	'''
	# Functionality
		Mount google drive. Since colab does not save files, we want to make it easier to directly access files in google drive.
	# Arguments
		Nothing
	# Returns
		drive_root: the working directory mounted
	'''
	mount_directory = "/content/gdrive"
	drive = colab.drive
	drive.mount(mount_directory, force_remount=True)
	drive_root = mount_directory + "/" + list(filter(lambda x: x[0] != '.', os.listdir(mount_directory)))[0]
	return drive_root

def get_all_files_from_dir(directory):

    '''
    # Functionality
    Get the whole list of files in a folder
	# Arguments
		directory: string.  (e.g.,Users/laowang/~vgg/data/flowers/102/102flowers.tgz)
	# Returns
		None
	'''
    file_paths = []
    for root, dirs, files in os.walk(directory):
        file_paths += [os.path.join(root, x) for x in files]
    return sorted(file_paths)

In [4]:
ROOT_DIR =  mount_google_drive() + "/ECE457B-Project/"
DATASET_DIR = ROOT_DIR + "sec8ks/"
print(DATASET_DIR)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/ECE457B-Project/sec8ks/


## Retrieval from Internet

In [0]:
downloader = sec_edgar_downloader.Downloader(DATASET_DIR)

In [199]:
downloader.get("8-K", "GOOGL", after_date="20120103", before_date="20161230")

13

## Parse 8-k reports

In [0]:
# file_name = DATASET_DIR + "sec_edgar_filings/GOOGL/8-K/0001652044-16-000035.txt"

In [0]:
# with open(file_name, "r") as f:
#     soup = BeautifulSoup(f.read(), "html")
#     found = False
# date = str(soup.find("acceptance-datetime"))[21:29]
# date

In [0]:
# for table in soup.findAll('table'):
#   if str(table).find("Financial Statements and Exhibits") != -1:
#     found = True

#   if found:
#     if str(table).find("Description") != -1: # Normally Item description
#       sample = pd.read_html(str(table))[0]
#       print(list(sample[1].dropna()[1:]), date)
#       break

In [0]:
def extract_fin_info(file_name):
  with open(file_name, "r") as f:
    soup = BeautifulSoup(f.read(), "html")
    found = False

  date = str(soup.find("acceptance-datetime"))[21:29]

  for table in soup.findAll('table'):
    if str(table).find("Financial Statements and Exhibits") != -1:
      found = True

    if found:
      if str(table).find("Description") != -1: # Normally Item description
        sample = pd.read_html(str(table))[0]
        return list(sample[1].dropna()[1:]), date

In [31]:
results = []
dates = []

for file_name in get_all_files_from_dir(DATASET_DIR):
  try:
    result, date = extract_fin_info(file_name)
  except:
    print(file_name)
    pass
  if result:
    results.append(result)
    dates.append(date)

/content/gdrive/My Drive/ECE457B-Project/sec8ks/sec_edgar_filings/GOOGL/8-K/0001652044-15-000007.txt


In [32]:
len(get_all_files_from_dir(DATASET_DIR))

13

In [33]:
len(results)

7

In [35]:
results

[['Press release of Google Inc. dated October 22, 2015'],
 ['Press release of Google Inc. dated October 22, 2015'],
 ['Press release of Alphabet Inc. dated February 1, 2016'],
 ['Press release of Alphabet Inc. dated April 21, 2016',
  'Financial tables for the quarter ended March 31, 2016'],
 ['Consent of Ernst & Young LLP, Independent Registered Public Accounting Firm',
  'Updates, where applicable, to Part II, Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations, from Alphabet’s Annual Report on Form 10-K for the year ended December 31, 2015, as filed with the Securities and Exchange Commission on February 11, 2016',
  'Updates, where applicable, to Part II, Item 8. Financial Statements and Supplementary Data, from Alphabet’s Annual Report on Form 10-K for the year ended December 31, 2015, as filed with the Securities and Exchange Commission on February 11, 2016',
  'XBRL Instance Document',
  'XBRL Taxonomy Extension Schema Document',
  'XBRL

In [198]:
dates[1]

'20120604'